## Diet problem

A canteen has to plan the composition of the meals that it provides. A meal can be composed of the types of food indicated in the following table. Costs, in Euro per hg, and availabilities, in hg, are also indicated.

Food|Cost|Availability
----|----|------------
Bread|0.1|4
Milk|0.5|3
Eggs|0.12|1
Meat|0.9|2
Cake|1.3|2

A meal must contain at least the following amount of each nutrient

Nutrient|Minimal quantity
--------|----------------
Calories|600 cal
Proteins|50 g
Calcium|0.7 g

Each hg of each type of food contains to following amount of nutrients

Food|Calories|Proteins|Calcium
----|--------|--------|-------
Bread|30 cal|5 g|0.02 g
Milk|50 cal|15 g|0.15 g
Eggs|150 cal|30 g|0.05 g
Meat|180 cal|90 g|0.08 g
Cake|400 cal|70 g|0.01 g

Give a linear programming formulation for the problem of finding a diet (a meal) of minimum total cost which satisfies the minimum nutrient requirements.

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install mip

In [2]:
# We need to import the package mip
import mip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#SETS
# Food, set of foods
I = ['Bread', 'Milk', 'Eggs', 'Meat', 'Cake']

# Nutrients, set of nutrients
J = {'Calories','Proteins', 'Calcium'}

#Parameters:

# Cost in Euro per hg of food ; c is included in/belongs to set I
c = {'Bread':0.1, 'Milk': 0.5, 'Eggs':0.12, 'Meat':0.9, 'Cake':1.3 }
#c =[0.1, 0.5, 0.12, 0.9, 1.3]

# Availability per hg of food; q is included in set I
q = {'Bread':4, 'Milk': 3, 'Eggs':1, 'Meat':2, 'Cake':2 }

# minum nutrients ; minimum requirements bi, i belongs to set J
b = {'Calories':600, 'Proteins':50, 'Calcium':0.7 }

# Nutrients per hf of food; a belongs to J
a = {('Bread','Calories'):30,('Milk','Calories'):50,('Eggs','Calories'):150,('Meat','Calories'):180,('Cake','Calories'):400,
('Bread','Proteins'):5,('Milk','Proteins'):15,('Eggs','Proteins'):30,('Meat','Proteins'):90,('Cake','Proteins'):70,
('Bread','Calcium'):0.02,('Milk','Calcium'):0.15,('Eggs','Calcium'):0.05,('Meat','Calcium'):0.08,('Cake','Calcium'):0.01}

#DECISION VARIABLES:
#xi = amount of food, i belongs to I

#COSTRAINTS:

#minimum quantity c[i]*x[i]
#sum (a[i][j]) for every j belonging to J >= b[j] for each j belonging to J  <--- minimum requirement costraint

#availability of food
#  x[i]<=q[i] for each i belonging to I
#with x[i] >= 0 (continuous and bigger than zero)

#We'll use dictionaries in which the keys are the names

Now we create an empty model and add the variables

In [ ]:
# Define a model
model = mip.Model()

# Define variables
x = { i :model.add_var(name = i) for i in I} #x is a dictionary
#add_var(name='', lb=0.0, ub=inf, obj=0.0, var_type='C', column=None)
#Creates a new variable in the model, returning its reference

#Parameters
#name (str) – variable name (optional)
#lb (numbers.Real) – variable lower bound, default 0.0
#ub (numbers.Real) – variable upper bound, default infinity

#examples
#The following code adds a vector of binary variables x[0], ..., x[n-1] to the model m:

#x = [m.add_var(var_type=BINARY) for i in range(n)]

In [ ]:
print (x)

{'Bread': <mip.entities.Var object at 0x7e67e9ffc310>, 'Milk': <mip.entities.Var object at 0x7e67e9ffc1c0>, 'Eggs': <mip.entities.Var object at 0x7e67e9ffef50>, 'Meat': <mip.entities.Var object at 0x7e67e9ffc190>, 'Cake': <mip.entities.Var object at 0x7e67e9ffee60>}


Let's write the objective function: in the general case, it can be written as a sum over the set of models.

In [ ]:
# Define the objective function
model.objective = mip.minimize(mip.xsum(c[i]*x[i] for i in I))

The constraints can be generated in loops:

In [ ]:
# CONSTRAINTS

# Availability constraint
for i,food in enumerate(I): # i is the index, food is the key of the dictionary
  model.add_constr(x[food]<=q[food])

# Minum nutrients
for j in J:
  model.add_constr(mip.xsum(a[i, j]*x[i] for i in I)>=b[j])
  #mip.xsum() >=b == sommatoria >= del valor b

The model is complete. Let's solve it and print the optimal solution.

In [ ]:
# Optimizing command
model.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [ ]:
# Optimal objective function value
model.objective.x

3.37

In [ ]:
# Printing the variables values
for i in model.vars:
  print(i.name)
  print(i.x)

Bread
3.9999999999999996
Milk
3.0
Eggs
1.0
Meat
1.5000000000000002
Cake
0.0


## Oil blending problem

A refinery has to blend 4 types of oil to obtain 3 types of gasoline. The following table reports the available quantity of oil of each type (in barrels) and the respective unit cost (Euro per barrel)

Oil type|Cost|Availability
--------|----|------------
1|9|5000
2|7|2400
3|12|4000
4|6|1500


Blending constraints are to be taken into account, since each type of gasoline must contain at least a specific, predefined, quantity of each type of oil, as indicated in the next table. The unit revenue of each type of gasoline (Euro per barrel) is also indicated

Gasoline type|Requirements|Revenue
-------------|------------|-------
A|$\geq$ 20% of type 2| 12
A|$\leq$ 30% of type 3|12
B|$\geq$ 40% of type 3|18
C|$\leq$ 50% of type 2|10


In [18]:
#E' lo stesso problema di esercitazione 1.2 ma con più dati -> applica le stesse formule
# Set of oil types
#I =range(4) # dà una lista di elementi da 0 a 3 (4 escluso)
I = range (1,5) #MA gli oli disponibili sono chiamati 1 2 3 4

# Set of gasoline types
J =['A', 'B', 'C']

# Unit cost for oil of type i
c = {1: 9, 2 : 7, 3:12, 4:6}

# Availability of oil type i
b ={1:5000, 2:2400 , 3:4000 , 4:1500}

# Price of gasoline of type j
r ={'A':12, 'B': 18, 'C': 10}

# Maximum quantity (percentage) of oil
q_max = {}
for i in I:
  for j in J:
    q_max[(str(i),j)] = 1
q_max[('2','A')] = 0.3
q_max[('1','C')] = 0.5

# Minimum quantity (percentage) of oil
q_min = {}
for i in I:
  for j in J:
    q_min[(str(i),j)] = 0
q_min[('1','A')] = 0.2
q_min[('2','B')] = 0.4

Now we create an empty model and add the variables

In [19]:
# Define a model
model2 = mip.Model()

# Define variables
#x = {(str(i), j): model2.add_var(name = str)} #amount of oil i usato nel j-esimo gasolio
#ho creato una tabella le cui chiavi sono la coppia di indici srt(i), j (entrambi stringhe -> i è un intero ma str(i) è una stringa)
x = {(str(i), j): model2.add_var(name=str(i)+j) for i in I for j in J} # la stringa dev'essere UNICA!!
y = { j :model2.add_var(name = j) for j in J} #amount of j-esimo gasolina prodotta

In [20]:
# Define the objective function
#print(*I, sep =" ")
oo = mip.xsum(r[j]*y[j] for j in J)
uu = mip.xsum(c[i]*x[str(i), j] for i in I for j in J )
print(oo)
print(uu)
print(oo-uu)
model2.objective = mip.maximize( oo - uu )



+ 12A + 18B + 10C 
+ 91A + 91B + 91C + 72A + 72B + 72C + 123A + 123B + 123C + 64A + 64B + 64C 
+ 12A + 18B + 10C - 91A - 91B - 91C - 72A - 72B - 72C - 123A - 123B - 123C - 64A - 64B - 64C 


In [21]:
# CONSTRAINTS
# Availability constraint
for i in I:
  model2.add_constr(mip.xsum(x[str(i), j]for j in J) <= b[i])# for i in I)

# Conservation constraint
for j in J:
  model2.add_constr(mip.xsum(x[str(i),j ] for i in I) == y[j])

# Maximum quantity
for i in I:
  for j in J:
    model2.add_constr(x[str(i), j] <= q_max[str(i), j]*y[j])

# Minimum quantity
for i in I:
  for j in J:
    model2.add_constr(x[str(i), j] >= q_min[str(i), j]*y[j])

The model is complete. Let's solve it and print the optimal solution.

In [22]:
# Optimizing command
model2.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [23]:
# Optimal objective function value
model2.objective.x

72000.00000000001

In [24]:
# Printing the variables values
for i in model2.vars:
  print("Oil and gasoline:"+i.name+' amount '+str(i.x))
 # print(i.x)
  #print('-----')


Oil and gasoline:1A amount 5000.0
Oil and gasoline:1B amount 0.0
Oil and gasoline:1C amount 0.0
Oil and gasoline:2A amount 0.0
Oil and gasoline:2B amount 2400.0
Oil and gasoline:2C amount 0.0
Oil and gasoline:3A amount 1900.0
Oil and gasoline:3B amount 2100.0
Oil and gasoline:3C amount 0.0
Oil and gasoline:4A amount 0.0
Oil and gasoline:4B amount 1500.0
Oil and gasoline:4C amount 0.0
Oil and gasoline:A amount 6900.000000000001
Oil and gasoline:B amount 6000.0
Oil and gasoline:C amount 0.0
